In [ ]:
import os, io
import pytz
import urllib, json
import datetime as dt
import pandas as pd
import csv
from datetime import date, datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from sqlalchemy.sql import select, and_
from settings.db import WAREHOSUE_URL
from settings.apis import FORECAST_API_SETTINGS

engine_warehouse = create_engine(WAREHOSUE_URL, poolclass=NullPool)
api_base_url = f"{FORECAST_API_SETTINGS['host']}:{FORECAST_API_SETTINGS['port']}/api/forecast/"
api_headers = {FORECAST_API_SETTINGS['api_key_name']: FORECAST_API_SETTINGS['api_key_value']}

In [ ]:
#Select here site to get forecasts for
from settings.sites import kherson as sites_list

query = f'''SELECT site_name, w_code FROM dim_site WHERE site_name in ({str(sites_list)[1:-1]})'''
print(query)
with engine_warehouse.connect() as connection:
    sites = dict(connection.execute(query).fetchall())

In [ ]:
#Set here forecast type and dates
forecast_type = 'applied/final'

dates_list = pd.date_range(start='2022-10-1', end='2022-10-1').to_list()


In [ ]:
target_folder = 'data/forecasts/{}_{}/'.format(dates_list[0].strftime('%Y-%m-%d'), dates_list[-1].strftime('%Y-%m-%d'))
print(target_folder)
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

In [ ]:
forecasts = dict()

for site, w_code in sites.items():
    forecasts.update({site: list()})
    for date in dates_list:
        date_str = date.strftime('%Y-%m-%d')
        api_url = api_base_url + f"{forecast_type}/{w_code}/{date_str}"
        request = urllib.request.Request(api_url, headers=api_headers, method='GET')
        with urllib.request.urlopen(request) as response:
            data = json.loads(response.read().decode('utf-8'))['final_forecast']
            forecast = pd.DataFrame(index=range(1, len(data['forecast_values']) + 1), data=data['forecast_values'], columns=[date_str])
            forecasts[site].append(forecast)
    forecasts.update({site: pd.concat(forecasts[site], axis=1)})
    with pd.ExcelWriter(target_folder + f'{site}_forecast.xlsx', engine="openpyxl") as  writer:
        forecasts[site].to_excel(writer, 'results_daily')

## UTP Makets

In [ ]:
def get_fake_data_API(w_code : str, date : date):
    date_str = date.strftime('%Y-%m-%d')
    api_url = api_base_url + f"{forecast_type}/{w_code}/{date_str}"
    request = urllib.request.Request(api_url, headers=api_headers, method='GET')
    with urllib.request.urlopen(request) as response:
        data = json.loads(response.read().decode('utf-8'))['final_forecast']
        forecast = pd.DataFrame(index=range(1, len(data['forecast_values']) + 1), data=data['forecast_values'], columns=[date_str])

    forecast['{}'.format(date_str)]
    forecast['Generation']=forecast['{}'.format(date_str)]
    num = forecast['Generation']._get_numeric_data()
    num[num < 0] = 0
    generation1 = forecast['Generation'].values.tolist()
    forecast['Consumption']=forecast['{}'.format(date_str)]*(-1)
    numb = forecast['Consumption']._get_numeric_data()
    numb[numb < 0] = 0
    consumption = forecast['Consumption']
    consumption = forecast['Consumption'].values.tolist()
    generation = generation1
    consurpation = consumption           
    return (generation, consurpation)


def create_row_list(sites : list, date : date):
    row_list = list()
    row_list.append(['((//30817', '{}'.format(date.strftime('%m%d')), '38269062', '++'])
    for code in sites:
        row_data = get_fake_data_API(w_code= code, date= date)
        consurption_list = ['({}_1)'.format(code)] + [sum(row_data[1])] + row_data[1] + ['']
        row_list.append(consurption_list)        
        generation_list = ['({}_2)'.format(code)] + [sum(row_data[0])] + row_data[0] + ['']
        row_list.append(generation_list)
    row_list.append(['==))'])


    return row_list
enter_date = dt.date(2023, 1, 30)

row_list = create_row_list(sites = sites.values(), date= enter_date)

with open('./data/utp_mock-up/forecast_{}_{}.txt'.format('kherson', enter_date.strftime('%m%d')), 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE, delimiter=':')
    writer.writerows(row_list)